# 모듈 임포트

In [124]:
import pandas as pd
import numpy as np
from gensim.models import word2vec, KeyedVectors, Word2Vec
import logging

In [125]:
# pip install gensim==3.8.3 = > ?

practiced_model = Word2Vec.load('word2vec_청와대')

2021-08-13 16:19:47,149 : INFO : loading Word2Vec object from word2vec_청와대
2021-08-13 16:19:47,199 : INFO : loading wv recursively from word2vec_청와대.wv.* with mmap=None
2021-08-13 16:19:47,199 : INFO : setting ignored attribute cum_table to None
2021-08-13 16:19:47,425 : INFO : Word2Vec lifecycle event {'fname': 'word2vec_청와대', 'datetime': '2021-08-13T16:19:47.425707', 'gensim': '4.0.1', 'python': '3.9.6 (v3.9.6:db3ff76da1, Jun 28 2021, 11:49:53) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}


# 국회 데이터 로드

In [126]:
assembly = pd.read_excel('assembly preprocessing.xlsx')
assembly = assembly.dropna(axis = 0)

In [127]:
display(assembly)

,title,count,categ,start,text
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['국가', '보안법', '폐지', '올라오', '자유', '대한민국', '월남',..."
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', '구현', '제재', '가하', '규정', '만들', '외의', '구현..."
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['사회권', '위원회', '국가', '인권', '위원회', '포괄적', '차별',..."
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연..."
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '..."
...,...,...,...,...,...
215,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,"['한국', '심리', '학회', '상담', '심리', '학회', '임상', '심리..."
216,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,"['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',..."
217,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,"['일요일', '공휴일', '휴일', '일괄적', '월요일', '정하', '의견',..."
218,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,"['순기능', '역효과', '일으키', '셧다', '운제', '청소년', '보호법'..."


## 국회 카테고리 확인

In [128]:
print(assembly['categ'].unique())
assembly['categ'].nunique()

['수사/법무/사법제도' '과학기술/정보통신' '인권/성평등/노동' '재난/안전/환경' '정치/선거/국회운영' '교육'
 '농업/임업/수산업/축산업' '보건의료' '외교/통일/국방/안보' '저출산/고령화/아동/청소년/가족' '국토/해양/교통'
 '행정/지방자치' '기타' '문화/체육/관광/언론' '재정/세제/금융/예산' '복지/보훈' '소비자/공정거래' '산업/통상']


18

## dataframe['text'] 필요한 형태로 가공

In [129]:
tokn_assembly = assembly['text']
txt_assembly = tokn_assembly.str.lstrip("['").str.rstrip("']").str.split("', '").to_list()
txt_assembly[0][0]

'국가'

# 국회

## 로그 산출 및 word2vec 모델 생성

In [130]:
%%time

# word2vec 모델학습에 로그 산출
logging.basicConfig(
    format = '%(asctime)s : %(levelname)s : %(message)s',
    level = logging.INFO)

# 초기화 및 모델학습
# sg = 0 => cbow / sg = 1 => skip-gram 
model_fname = 'word2vec_국회'
model = word2vec.Word2Vec(sentences = txt_assembly,
                          min_count = 3,
                          window = 4,
                          sg = 0,
                         vector_size = 200)
model.save(model_fname)

2021-08-13 16:19:47,515 : INFO : collecting all words and their counts
2021-08-13 16:19:47,516 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-13 16:19:47,524 : INFO : collected 7520 word types from a corpus of 51798 raw words and 220 sentences
2021-08-13 16:19:47,525 : INFO : Creating a fresh vocabulary
2021-08-13 16:19:47,535 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 3347 unique words (44.50797872340426%% of original 7520, drops 4173)', 'datetime': '2021-08-13T16:19:47.535754', 'gensim': '4.0.1', 'python': '3.9.6 (v3.9.6:db3ff76da1, Jun 28 2021, 11:49:53) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-08-13 16:19:47,536 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 leaves 46496 word corpus (89.76408355534963%% of original 51798, drops 5302)', 'datetime': '2021-08-13T16:19:47.536360', 'gensim': '4.0.1', 'python': '3.9.6 (v3.9.6:db3ff76da1, 

CPU times: user 433 ms, sys: 16.7 ms, total: 450 ms
Wall time: 239 ms


## 국회 Word2vec 결과

In [131]:
len(model_assembly.wv)

3347

In [132]:
word2vec_assembly = model_assembly.wv
words_assembly = list(word2vec_assembly.index_to_key)

In [133]:
words_assembly[:30]

['국민',
 '국가',
 '처벌',
 '여성',
 '피해자',
 '투표',
 '범죄',
 '개정',
 '대한민국',
 '보호',
 '법률',
 '제조',
 '사건',
 '가족',
 '선거',
 '국회',
 '헌법',
 '성범죄',
 '피해',
 '행위',
 '사람',
 '문제',
 '현재',
 '발생',
 '시키',
 '모든',
 '만들',
 '청소년',
 '코로나',
 '사전']

## 국회 word2vec '슬픔'과의 유사도 계산

In [134]:
print(word2vec_assembly.key_to_index['슬픔'])
word2vec_assembly.most_similar('슬픔')

KeyError: '슬픔'

## 국회 word2vec '분노'와의 유사도 계산

In [135]:
print(word2vec_assembly.key_to_index['분노'])
word2vec_assembly.most_similar('분노')

893


[('다시', 0.9978609085083008),
 ('조사', 0.9978427886962891),
 ('적용', 0.9978371262550354),
 ('필요', 0.9978361129760742),
 ('국가', 0.9978320598602295),
 ('개선', 0.997826874256134),
 ('처벌', 0.9978240132331848),
 ('의원', 0.9978235363960266),
 ('당하', 0.9978234767913818),
 ('검사', 0.9978168606758118)]

In [136]:
# 'error' 가 존재하는 row => 임베딩 벡터가 존재하지 않아 단어 유사도 계산 불가
similarity = []
average = []

for i in range(len(txt_assembly)):
    similarity_in = []
    for j in range(len(txt_assembly[i])):
        try:
            similarity_in.append(word2vec_assembly.similarity(w1 = txt_assembly[i][j], w2 = '분노'))
        except KeyError:
            print(f'{i}번째 문서의 {j}번째에서 error')
            similarity_in.append(0)
    similarity.append(similarity_in)
    average.append(np.mean(similarity[i]))
            

0번째 문서의 8번째에서 error
0번째 문서의 12번째에서 error
0번째 문서의 16번째에서 error
0번째 문서의 20번째에서 error
0번째 문서의 23번째에서 error
0번째 문서의 24번째에서 error
0번째 문서의 38번째에서 error
0번째 문서의 45번째에서 error
0번째 문서의 53번째에서 error
0번째 문서의 54번째에서 error
0번째 문서의 66번째에서 error
0번째 문서의 73번째에서 error
0번째 문서의 76번째에서 error
0번째 문서의 82번째에서 error
0번째 문서의 89번째에서 error
0번째 문서의 100번째에서 error
0번째 문서의 101번째에서 error
0번째 문서의 104번째에서 error
0번째 문서의 105번째에서 error
0번째 문서의 113번째에서 error
0번째 문서의 114번째에서 error
0번째 문서의 118번째에서 error
0번째 문서의 126번째에서 error
0번째 문서의 127번째에서 error
0번째 문서의 132번째에서 error
0번째 문서의 136번째에서 error
0번째 문서의 139번째에서 error
0번째 문서의 141번째에서 error
0번째 문서의 142번째에서 error
0번째 문서의 143번째에서 error
0번째 문서의 159번째에서 error
0번째 문서의 167번째에서 error
0번째 문서의 170번째에서 error
0번째 문서의 182번째에서 error
0번째 문서의 184번째에서 error
0번째 문서의 191번째에서 error
0번째 문서의 199번째에서 error
0번째 문서의 203번째에서 error
0번째 문서의 212번째에서 error
0번째 문서의 213번째에서 error
0번째 문서의 218번째에서 error
0번째 문서의 219번째에서 error
0번째 문서의 221번째에서 error
0번째 문서의 222번째에서 error
0번째 문서의 234번째에서 error
0번째 문서의 235번째에서 error
0번째 

15번째 문서의 97번째에서 error
15번째 문서의 113번째에서 error
15번째 문서의 120번째에서 error
15번째 문서의 147번째에서 error
15번째 문서의 151번째에서 error
15번째 문서의 156번째에서 error
15번째 문서의 161번째에서 error
15번째 문서의 165번째에서 error
15번째 문서의 166번째에서 error
15번째 문서의 167번째에서 error
15번째 문서의 191번째에서 error
15번째 문서의 211번째에서 error
15번째 문서의 216번째에서 error
15번째 문서의 218번째에서 error
15번째 문서의 219번째에서 error
15번째 문서의 229번째에서 error
16번째 문서의 4번째에서 error
16번째 문서의 11번째에서 error
16번째 문서의 12번째에서 error
16번째 문서의 22번째에서 error
16번째 문서의 29번째에서 error
16번째 문서의 37번째에서 error
16번째 문서의 51번째에서 error
16번째 문서의 54번째에서 error
16번째 문서의 60번째에서 error
16번째 문서의 62번째에서 error
16번째 문서의 63번째에서 error
16번째 문서의 64번째에서 error
16번째 문서의 65번째에서 error
16번째 문서의 67번째에서 error
16번째 문서의 72번째에서 error
16번째 문서의 74번째에서 error
16번째 문서의 103번째에서 error
16번째 문서의 105번째에서 error
16번째 문서의 143번째에서 error
16번째 문서의 144번째에서 error
16번째 문서의 147번째에서 error
16번째 문서의 151번째에서 error
16번째 문서의 174번째에서 error
16번째 문서의 200번째에서 error
17번째 문서의 5번째에서 error
17번째 문서의 10번째에서 error
17번째 문서의 43번째에서 error
17번째 문서의 69번째에서 error
17번째 문서의 74

40번째 문서의 401번째에서 error
41번째 문서의 0번째에서 error
41번째 문서의 3번째에서 error
41번째 문서의 20번째에서 error
41번째 문서의 21번째에서 error
41번째 문서의 36번째에서 error
41번째 문서의 51번째에서 error
41번째 문서의 54번째에서 error
41번째 문서의 69번째에서 error
41번째 문서의 99번째에서 error
41번째 문서의 104번째에서 error
41번째 문서의 105번째에서 error
41번째 문서의 108번째에서 error
41번째 문서의 109번째에서 error
41번째 문서의 112번째에서 error
41번째 문서의 113번째에서 error
41번째 문서의 120번째에서 error
41번째 문서의 126번째에서 error
41번째 문서의 164번째에서 error
41번째 문서의 171번째에서 error
41번째 문서의 176번째에서 error
41번째 문서의 178번째에서 error
41번째 문서의 199번째에서 error
41번째 문서의 208번째에서 error
41번째 문서의 215번째에서 error
41번째 문서의 219번째에서 error
41번째 문서의 226번째에서 error
41번째 문서의 230번째에서 error
41번째 문서의 235번째에서 error
41번째 문서의 236번째에서 error
41번째 문서의 247번째에서 error
41번째 문서의 251번째에서 error
41번째 문서의 256번째에서 error
42번째 문서의 38번째에서 error
42번째 문서의 41번째에서 error
42번째 문서의 43번째에서 error
42번째 문서의 47번째에서 error
42번째 문서의 54번째에서 error
42번째 문서의 58번째에서 error
42번째 문서의 65번째에서 error
42번째 문서의 74번째에서 error
42번째 문서의 77번째에서 error
42번째 문서의 92번째에서 error
42번째 문서의 105번째에서 error
42번째 문서의 

81번째 문서의 186번째에서 error
81번째 문서의 197번째에서 error
81번째 문서의 210번째에서 error
81번째 문서의 211번째에서 error
81번째 문서의 222번째에서 error
81번째 문서의 228번째에서 error
81번째 문서의 252번째에서 error
81번째 문서의 282번째에서 error
81번째 문서의 284번째에서 error
81번째 문서의 286번째에서 error
81번째 문서의 297번째에서 error
81번째 문서의 311번째에서 error
82번째 문서의 16번째에서 error
82번째 문서의 17번째에서 error
82번째 문서의 20번째에서 error
82번째 문서의 49번째에서 error
83번째 문서의 6번째에서 error
83번째 문서의 14번째에서 error
83번째 문서의 15번째에서 error
83번째 문서의 35번째에서 error
83번째 문서의 36번째에서 error
83번째 문서의 44번째에서 error
83번째 문서의 47번째에서 error
83번째 문서의 53번째에서 error
83번째 문서의 55번째에서 error
83번째 문서의 56번째에서 error
83번째 문서의 61번째에서 error
83번째 문서의 64번째에서 error
83번째 문서의 65번째에서 error
83번째 문서의 66번째에서 error
83번째 문서의 77번째에서 error
83번째 문서의 90번째에서 error
83번째 문서의 92번째에서 error
83번째 문서의 106번째에서 error
83번째 문서의 108번째에서 error
83번째 문서의 109번째에서 error
83번째 문서의 113번째에서 error
83번째 문서의 116번째에서 error
83번째 문서의 118번째에서 error
83번째 문서의 124번째에서 error
83번째 문서의 125번째에서 error
83번째 문서의 136번째에서 error
83번째 문서의 142번째에서 error
83번째 문서의 144번째에서 error
83번째 문서의 1

98번째 문서의 10번째에서 error
98번째 문서의 104번째에서 error
98번째 문서의 111번째에서 error
98번째 문서의 124번째에서 error
98번째 문서의 142번째에서 error
98번째 문서의 171번째에서 error
98번째 문서의 175번째에서 error
99번째 문서의 0번째에서 error
99번째 문서의 4번째에서 error
99번째 문서의 13번째에서 error
99번째 문서의 25번째에서 error
99번째 문서의 28번째에서 error
99번째 문서의 38번째에서 error
99번째 문서의 40번째에서 error
99번째 문서의 42번째에서 error
99번째 문서의 43번째에서 error
99번째 문서의 46번째에서 error
99번째 문서의 47번째에서 error
99번째 문서의 54번째에서 error
99번째 문서의 55번째에서 error
99번째 문서의 56번째에서 error
99번째 문서의 58번째에서 error
99번째 문서의 63번째에서 error
99번째 문서의 73번째에서 error
99번째 문서의 76번째에서 error
99번째 문서의 78번째에서 error
99번째 문서의 84번째에서 error
99번째 문서의 88번째에서 error
99번째 문서의 89번째에서 error
99번째 문서의 90번째에서 error
99번째 문서의 95번째에서 error
99번째 문서의 97번째에서 error
99번째 문서의 98번째에서 error
99번째 문서의 102번째에서 error
99번째 문서의 114번째에서 error
99번째 문서의 115번째에서 error
99번째 문서의 118번째에서 error
99번째 문서의 119번째에서 error
99번째 문서의 126번째에서 error
99번째 문서의 143번째에서 error
99번째 문서의 145번째에서 error
99번째 문서의 149번째에서 error
99번째 문서의 152번째에서 error
99번째 문서의 165번째에서 error
99번째 문서의 169번째에서 

...

143번째 문서의 178번째에서 error
143번째 문서의 258번째에서 error
144번째 문서의 25번째에서 error
144번째 문서의 26번째에서 error
144번째 문서의 97번째에서 error
144번째 문서의 110번째에서 error
144번째 문서의 145번째에서 error
144번째 문서의 165번째에서 error
144번째 문서의 166번째에서 error
144번째 문서의 192번째에서 error
144번째 문서의 221번째에서 error
144번째 문서의 228번째에서 error
144번째 문서의 231번째에서 error
144번째 문서의 241번째에서 error
144번째 문서의 279번째에서 error
144번째 문서의 280번째에서 error
144번째 문서의 330번째에서 error
145번째 문서의 40번째에서 error
145번째 문서의 53번째에서 error
145번째 문서의 61번째에서 error
145번째 문서의 82번째에서 error
145번째 문서의 89번째에서 error
145번째 문서의 90번째에서 error
145번째 문서의 106번째에서 error
145번째 문서의 128번째에서 error
145번째 문서의 130번째에서 error
145번째 문서의 132번째에서 error
145번째 문서의 135번째에서 error
145번째 문서의 164번째에서 error
145번째 문서의 166번째에서 error
145번째 문서의 167번째에서 error
145번째 문서의 168번째에서 error
145번째 문서의 174번째에서 error
145번째 문서의 177번째에서 error
145번째 문서의 181번째에서 error
145번째 문서의 185번째에서 error
145번째 문서의 186번째에서 error
145번째 문서의 187번째에서 error
145번째 문서의 190번째에서 error
145번째 문서의 191번째에서 error
145번째 문서의 192번째에서 error
145번째 문서의 208번째에서 error
1

161번째 문서의 47번째에서 error
161번째 문서의 48번째에서 error
161번째 문서의 50번째에서 error
161번째 문서의 73번째에서 error
161번째 문서의 79번째에서 error
161번째 문서의 82번째에서 error
161번째 문서의 88번째에서 error
161번째 문서의 109번째에서 error
161번째 문서의 123번째에서 error
161번째 문서의 130번째에서 error
161번째 문서의 146번째에서 error
161번째 문서의 160번째에서 error
161번째 문서의 166번째에서 error
161번째 문서의 176번째에서 error
161번째 문서의 181번째에서 error
161번째 문서의 188번째에서 error
161번째 문서의 191번째에서 error
161번째 문서의 195번째에서 error
161번째 문서의 213번째에서 error
161번째 문서의 218번째에서 error
161번째 문서의 220번째에서 error
161번째 문서의 226번째에서 error
161번째 문서의 236번째에서 error
161번째 문서의 250번째에서 error
161번째 문서의 264번째에서 error
161번째 문서의 265번째에서 error
161번째 문서의 268번째에서 error
161번째 문서의 283번째에서 error
161번째 문서의 288번째에서 error
161번째 문서의 289번째에서 error
161번째 문서의 293번째에서 error
161번째 문서의 296번째에서 error
161번째 문서의 306번째에서 error
161번째 문서의 313번째에서 error
162번째 문서의 7번째에서 error
162번째 문서의 9번째에서 error
162번째 문서의 17번째에서 error
162번째 문서의 25번째에서 error
162번째 문서의 28번째에서 error
162번째 문서의 39번째에서 error
162번째 문서의 48번째에서 error
162번째 문서의 62번째에서 error
162번째 문서의

181번째 문서의 188번째에서 error
181번째 문서의 191번째에서 error
181번째 문서의 199번째에서 error
181번째 문서의 225번째에서 error
181번째 문서의 226번째에서 error
181번째 문서의 245번째에서 error
181번째 문서의 246번째에서 error
181번째 문서의 247번째에서 error
181번째 문서의 251번째에서 error
181번째 문서의 254번째에서 error
181번째 문서의 259번째에서 error
181번째 문서의 269번째에서 error
181번째 문서의 270번째에서 error
181번째 문서의 271번째에서 error
181번째 문서의 275번째에서 error
181번째 문서의 276번째에서 error
181번째 문서의 279번째에서 error
181번째 문서의 287번째에서 error
181번째 문서의 289번째에서 error
181번째 문서의 296번째에서 error
181번째 문서의 304번째에서 error
181번째 문서의 305번째에서 error
181번째 문서의 319번째에서 error
182번째 문서의 28번째에서 error
182번째 문서의 35번째에서 error
182번째 문서의 65번째에서 error
182번째 문서의 75번째에서 error
182번째 문서의 93번째에서 error
182번째 문서의 101번째에서 error
182번째 문서의 106번째에서 error
182번째 문서의 108번째에서 error
182번째 문서의 113번째에서 error
182번째 문서의 125번째에서 error
182번째 문서의 130번째에서 error
182번째 문서의 131번째에서 error
182번째 문서의 139번째에서 error
182번째 문서의 140번째에서 error
182번째 문서의 156번째에서 error
182번째 문서의 159번째에서 error
182번째 문서의 185번째에서 error
182번째 문서의 198번째에서 error
182번째 문서의 202번째에서 err

203번째 문서의 9번째에서 error
203번째 문서의 12번째에서 error
203번째 문서의 16번째에서 error
203번째 문서의 17번째에서 error
203번째 문서의 21번째에서 error
203번째 문서의 38번째에서 error
203번째 문서의 65번째에서 error
203번째 문서의 68번째에서 error
203번째 문서의 71번째에서 error
203번째 문서의 84번째에서 error
203번째 문서의 121번째에서 error
203번째 문서의 134번째에서 error
203번째 문서의 135번째에서 error
203번째 문서의 137번째에서 error
203번째 문서의 144번째에서 error
203번째 문서의 146번째에서 error
203번째 문서의 148번째에서 error
203번째 문서의 169번째에서 error
203번째 문서의 179번째에서 error
203번째 문서의 187번째에서 error
203번째 문서의 191번째에서 error
203번째 문서의 200번째에서 error
203번째 문서의 201번째에서 error
203번째 문서의 202번째에서 error
203번째 문서의 203번째에서 error
203번째 문서의 211번째에서 error
203번째 문서의 219번째에서 error
203번째 문서의 229번째에서 error
203번째 문서의 241번째에서 error
203번째 문서의 250번째에서 error
203번째 문서의 253번째에서 error
203번째 문서의 258번째에서 error
203번째 문서의 260번째에서 error
203번째 문서의 268번째에서 error
203번째 문서의 269번째에서 error
203번째 문서의 271번째에서 error
203번째 문서의 273번째에서 error
203번째 문서의 288번째에서 error
203번째 문서의 292번째에서 error
203번째 문서의 314번째에서 error
204번째 문서의 6번째에서 error
204번째 문서의 10번째에서 error
204번째 

219번째 문서의 70번째에서 error
219번째 문서의 72번째에서 error
219번째 문서의 99번째에서 error
219번째 문서의 112번째에서 error
219번째 문서의 117번째에서 error
219번째 문서의 142번째에서 error
219번째 문서의 143번째에서 error
219번째 문서의 149번째에서 error
219번째 문서의 150번째에서 error
219번째 문서의 159번째에서 error


In [137]:
len(average)

220

In [138]:
assembly['분노 유사도'] = average

In [139]:
assembly

,title,count,categ,start,text,분노 유사도
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['국가', '보안법', '폐지', '올라오', '자유', '대한민국', '월남',...",0.806483
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', '구현', '제재', '가하', '규정', '만들', '외의', '구현...",0.922949
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['사회권', '위원회', '국가', '인권', '위원회', '포괄적', '차별',...",0.906792
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연...",0.896154
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '...",0.899737
...,...,...,...,...,...,...
215,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,"['한국', '심리', '학회', '상담', '심리', '학회', '임상', '심리...",0.906695
216,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,"['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',...",0.891317
217,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,"['일요일', '공휴일', '휴일', '일괄적', '월요일', '정하', '의견',...",0.749549
218,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,"['순기능', '역효과', '일으키', '셧다', '운제', '청소년', '보호법'...",0.885646


# 청와대로 훈련된 word2vec

## '슬픔'과의 유사도 계산

In [140]:
print(practiced_model.wv.key_to_index['슬픔'])

3654


In [141]:
# 'error' 가 존재하는 row => 임베딩 벡터가 존재하지 않아 단어 유사도 계산 불가
similarity = []
average = []

for i in range(len(txt_assembly)):
    similarity_in = []
    for j in range(len(txt_assembly[i])):
        try:
            similarity_in.append(practiced_model.wv.similarity(w1 = txt_assembly[i][j], w2 = '슬픔'))
        except KeyError:
            print(f'{i}번째 문서의 {j}번째에서 error')
            similarity_in.append(0)
    similarity.append(similarity_in)
    average.append(np.mean(similarity[i]))
            

0번째 문서의 105번째에서 error
0번째 문서의 132번째에서 error
0번째 문서의 184번째에서 error
2번째 문서의 188번째에서 error
2번째 문서의 192번째에서 error
2번째 문서의 325번째에서 error
3번째 문서의 122번째에서 error
4번째 문서의 0번째에서 error
4번째 문서의 21번째에서 error
4번째 문서의 25번째에서 error
4번째 문서의 26번째에서 error
4번째 문서의 34번째에서 error
4번째 문서의 35번째에서 error
4번째 문서의 81번째에서 error
4번째 문서의 135번째에서 error
4번째 문서의 146번째에서 error
4번째 문서의 147번째에서 error
4번째 문서의 148번째에서 error
4번째 문서의 156번째에서 error
4번째 문서의 157번째에서 error
4번째 문서의 183번째에서 error
4번째 문서의 184번째에서 error
4번째 문서의 200번째에서 error
4번째 문서의 222번째에서 error
4번째 문서의 227번째에서 error
4번째 문서의 230번째에서 error
4번째 문서의 236번째에서 error
4번째 문서의 243번째에서 error
5번째 문서의 157번째에서 error
5번째 문서의 225번째에서 error
6번째 문서의 183번째에서 error
9번째 문서의 454번째에서 error
10번째 문서의 214번째에서 error
11번째 문서의 96번째에서 error
11번째 문서의 100번째에서 error
12번째 문서의 46번째에서 error
12번째 문서의 309번째에서 error
13번째 문서의 220번째에서 error
15번째 문서의 120번째에서 error
16번째 문서의 62번째에서 error
16번째 문서의 67번째에서 error
18번째 문서의 73번째에서 error
18번째 문서의 119번째에서 error
18번째 문서의 189번째에서 error
18번째 문서의 200번째에서 error
18번째 문서의 2

176번째 문서의 3번째에서 error
176번째 문서의 23번째에서 error
176번째 문서의 149번째에서 error
178번째 문서의 166번째에서 error
178번째 문서의 244번째에서 error
178번째 문서의 298번째에서 error
179번째 문서의 171번째에서 error
180번째 문서의 27번째에서 error
181번째 문서의 199번째에서 error
181번째 문서의 304번째에서 error
181번째 문서의 319번째에서 error
182번째 문서의 293번째에서 error
184번째 문서의 75번째에서 error
186번째 문서의 9번째에서 error
186번째 문서의 15번째에서 error
186번째 문서의 134번째에서 error
187번째 문서의 70번째에서 error
188번째 문서의 30번째에서 error
188번째 문서의 42번째에서 error
188번째 문서의 113번째에서 error
188번째 문서의 171번째에서 error
188번째 문서의 190번째에서 error
189번째 문서의 94번째에서 error
189번째 문서의 153번째에서 error
189번째 문서의 214번째에서 error
189번째 문서의 226번째에서 error
192번째 문서의 0번째에서 error
192번째 문서의 36번째에서 error
192번째 문서의 37번째에서 error
193번째 문서의 99번째에서 error
193번째 문서의 136번째에서 error
194번째 문서의 107번째에서 error
195번째 문서의 26번째에서 error
195번째 문서의 30번째에서 error
195번째 문서의 84번째에서 error
195번째 문서의 232번째에서 error
196번째 문서의 34번째에서 error
196번째 문서의 41번째에서 error
196번째 문서의 65번째에서 error
196번째 문서의 91번째에서 error
196번째 문서의 101번째에서 error
198번째 문서의 7번째에서 error
198번째 문서의 12번째에서 e

In [142]:
assembly['슬픔 유사도'] = average

In [143]:
assembly

,title,count,categ,start,text,분노 유사도,슬픔 유사도
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['국가', '보안법', '폐지', '올라오', '자유', '대한민국', '월남',...",0.806483,0.097695
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', '구현', '제재', '가하', '규정', '만들', '외의', '구현...",0.922949,0.041638
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['사회권', '위원회', '국가', '인권', '위원회', '포괄적', '차별',...",0.906792,0.019631
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연...",0.896154,-0.020920
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '...",0.899737,-0.043951
...,...,...,...,...,...,...,...
215,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,"['한국', '심리', '학회', '상담', '심리', '학회', '임상', '심리...",0.906695,-0.040186
216,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,"['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',...",0.891317,-0.047650
217,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,"['일요일', '공휴일', '휴일', '일괄적', '월요일', '정하', '의견',...",0.749549,0.011573
218,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,"['순기능', '역효과', '일으키', '셧다', '운제', '청소년', '보호법'...",0.885646,0.017585


## '분노'와의 유사도 계산

In [144]:
print(practiced_model.wv.key_to_index['분노'])

751


In [145]:
# 'error' 가 존재하는 row => 임베딩 벡터가 존재하지 않아 단어 유사도 계산 불가
similarity = []
average = []

for i in range(len(txt_assembly)):
    similarity_in = []
    for j in range(len(txt_assembly[i])):
        try:
            similarity_in.append(practiced_model.wv.similarity(w1 = txt_assembly[i][j], w2 = '분노'))
        except KeyError:
            print(f'{i}번째 문서의 {j}번째에서 error')
            similarity_in.append(0)
    similarity.append(similarity_in)
    average.append(np.mean(similarity[i]))
            

0번째 문서의 105번째에서 error
0번째 문서의 132번째에서 error
0번째 문서의 184번째에서 error
2번째 문서의 188번째에서 error
2번째 문서의 192번째에서 error
2번째 문서의 325번째에서 error
3번째 문서의 122번째에서 error
4번째 문서의 0번째에서 error
4번째 문서의 21번째에서 error
4번째 문서의 25번째에서 error
4번째 문서의 26번째에서 error
4번째 문서의 34번째에서 error
4번째 문서의 35번째에서 error
4번째 문서의 81번째에서 error
4번째 문서의 135번째에서 error
4번째 문서의 146번째에서 error
4번째 문서의 147번째에서 error
4번째 문서의 148번째에서 error
4번째 문서의 156번째에서 error
4번째 문서의 157번째에서 error
4번째 문서의 183번째에서 error
4번째 문서의 184번째에서 error
4번째 문서의 200번째에서 error
4번째 문서의 222번째에서 error
4번째 문서의 227번째에서 error
4번째 문서의 230번째에서 error
4번째 문서의 236번째에서 error
4번째 문서의 243번째에서 error
5번째 문서의 157번째에서 error
5번째 문서의 225번째에서 error
6번째 문서의 183번째에서 error
9번째 문서의 454번째에서 error
10번째 문서의 214번째에서 error
11번째 문서의 96번째에서 error
11번째 문서의 100번째에서 error
12번째 문서의 46번째에서 error
12번째 문서의 309번째에서 error
13번째 문서의 220번째에서 error
15번째 문서의 120번째에서 error
16번째 문서의 62번째에서 error
16번째 문서의 67번째에서 error
18번째 문서의 73번째에서 error
18번째 문서의 119번째에서 error
18번째 문서의 189번째에서 error
18번째 문서의 200번째에서 error
18번째 문서의 2

176번째 문서의 3번째에서 error
176번째 문서의 23번째에서 error
176번째 문서의 149번째에서 error
178번째 문서의 166번째에서 error
178번째 문서의 244번째에서 error
178번째 문서의 298번째에서 error
179번째 문서의 171번째에서 error
180번째 문서의 27번째에서 error
181번째 문서의 199번째에서 error
181번째 문서의 304번째에서 error
181번째 문서의 319번째에서 error
182번째 문서의 293번째에서 error
184번째 문서의 75번째에서 error
186번째 문서의 9번째에서 error
186번째 문서의 15번째에서 error
186번째 문서의 134번째에서 error
187번째 문서의 70번째에서 error
188번째 문서의 30번째에서 error
188번째 문서의 42번째에서 error
188번째 문서의 113번째에서 error
188번째 문서의 171번째에서 error
188번째 문서의 190번째에서 error
189번째 문서의 94번째에서 error
189번째 문서의 153번째에서 error
189번째 문서의 214번째에서 error
189번째 문서의 226번째에서 error
192번째 문서의 0번째에서 error
192번째 문서의 36번째에서 error
192번째 문서의 37번째에서 error
193번째 문서의 99번째에서 error
193번째 문서의 136번째에서 error
194번째 문서의 107번째에서 error
195번째 문서의 26번째에서 error
195번째 문서의 30번째에서 error
195번째 문서의 84번째에서 error
195번째 문서의 232번째에서 error
196번째 문서의 34번째에서 error
196번째 문서의 41번째에서 error
196번째 문서의 65번째에서 error
196번째 문서의 91번째에서 error
196번째 문서의 101번째에서 error
198번째 문서의 7번째에서 error
198번째 문서의 12번째에서 e

In [146]:
len(average)

220

In [147]:
assembly['분노 유사도 - tr word2vec_청와대'] = average

In [148]:
assembly

,title,count,categ,start,text,분노 유사도,슬픔 유사도,분노 유사도 - tr word2vec_청와대
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['국가', '보안법', '폐지', '올라오', '자유', '대한민국', '월남',...",0.806483,0.097695,0.143033
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', '구현', '제재', '가하', '규정', '만들', '외의', '구현...",0.922949,0.041638,0.013914
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['사회권', '위원회', '국가', '인권', '위원회', '포괄적', '차별',...",0.906792,0.019631,0.071121
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연...",0.896154,-0.020920,-0.037472
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '...",0.899737,-0.043951,-0.045287
...,...,...,...,...,...,...,...,...
215,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,"['한국', '심리', '학회', '상담', '심리', '학회', '임상', '심리...",0.906695,-0.040186,-0.019991
216,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,"['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',...",0.891317,-0.047650,0.004244
217,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,"['일요일', '공휴일', '휴일', '일괄적', '월요일', '정하', '의견',...",0.749549,0.011573,-0.096007
218,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,"['순기능', '역효과', '일으키', '셧다', '운제', '청소년', '보호법'...",0.885646,0.017585,0.008036
